# Segmentation
___

## Contents 
### [1. Segmentation 기본 이론](#1)
* [1.1 segmentation의 접근방식](#1.1)
* [1.2 Image Segmentation의 과정](#1.2)

### [2. Pixel-wise Segmentation](#2) 
* [2.1 Pixel-wise Segmentation](#2.1)
    * [2.1.1 Threshold 설정하는 tip](#2.1.1)

### [3. Edge 기반 segmentation 1](#3)
* [3.1 Edge 기반의 Segmentation](#3.1)
    * [3.1.1 Gradient](#3.1.1)
    * [3.1.2 Edge 검출 필터](#3.1.2)
    * [3.1.3 edge 검출과 noise의 영향.](#3.1.3)
    * [3.1.4 Laplacian of Gaussian(LoG), Difference of Gaussian (DoG)](#3.1.4)

### [4. Edge 기반 segmentation 2](#4)
* [4.1 Edge thinning](#4.1)
* [4.2 Canny Edge Detector](#4.2)
    * [4.2.1 Canny Edge Detector의 상세과정 - Smoothing, Gradient 구하기](#4.2.1)
    * [4.2.2 Canny Edge Detector의 상세과정 - Thinning](#4.2.2)
    * [4.2.3 Canny Edge Detector의 상세과정 - Hysteresis](#4.2.3)
    * [4.2.4 Canny Edge 검출에서 **σ** 의 영향](#4.2.4)
    
    
* [4.3 SUSAN Edge Detector](#4.3)

### [5. region base Segmentation](#5)
* [5.1 Region을 정하는 방법](#5.1)
    * [5.1.1 Region Growing](#5.1.1)
    * [](#5.1.2)

___

# 1. Segmentation의 기본이론<a class="anchor" id="1"></a>
___

## 1.1 segmentation의 접근방식 <a class="anchor" id="1.1"></a>


#### 1) pixel-wise(픽셀 기반 방법)
- thresholding에 기반한 방식으로 histogram 을 이용해 pixel들의 분포를 확인한 후 적절한 threshold를 설정하고, pixel-wise 연산을 통해 pixel 별로 나누는 방식이며, 이진화에 많이 사용된다.

- Thresholding 으로는 전역(global) 혹은 지역(local)로 적용하는 영역에 따른 구분도 가능하고, 적응적(adaptive) 혹은 고정(fixed) 방식으로 경계값을 설정하는 방식에 따른 구별도 가능하다.

#### 2) Edge 기반
- edge 를 추출하는 필터를 사용하여 영상으로부터 경계를 추출하고, 흔히 non-maximum suppression 과 같은 방식을 사용하여 의미있는 edge와 의미없는 edge를 구별하는 방식을 사용한다.

#### 3) 영역 기반 
- pixel-wise 기반의 thresholding이나 edge 기반의 방식으로는 의미있는 영역을 구분하는 것이 쉽지 않고, 특히 잡음이 있는 환경에서 결과가 좋지 못한다.

- 영역 기반의 방법은 기본적으로 homogeneity(동질성)에 기반하고 있기 때문에 다른 방법들보다 의미 있는 영역으로 나누는데 적합하지만, homogeneity를 규정하는 rule을 어떻게 정할지가 관건이다.

- seed 라고 부르는 몇 개의 pixel에서 시작하여 영역을 넓혀가는 region growing 방식이 해당된다. 

- 이외에도 region merging, region splitting, split and merge, watershed 방식등도 있다.

___ 

</br>


## 1.2 Image Segmentation의 과정 <a class="anchor" id="1.2"></a>
- bottom-up 방식과 top-down 방식

#### 1) bottom-up 방식
- 비슷한 특징을 갖는 것들끼리 집단화 하는 것을 말한다

#### 2) top-down 방식
- 같은 객체에 해당하는 것들끼리 집단화 하는 것을 말한다.

![img1](./imgs/1.PNG)

___ 

</br>


# 2. Pixel-wise Segmentation <a class="anchor" id="2"></a>
___

## 2.1 Pixel-wise Segmentation <a class="anchor" id="2.1"></a>

#### Thresholding
- pixel-wise Segmentation 에서 흔히 사용되는 방식이 thresholding이다.

- 특정 임계값을 정하고 그보다 작으면 검은색 그보다 같거나 크면 흰색으로 구분하는 단순한 방법이다.

![img2](./imgs/2.PNG)

- 위이 이미지는 threshold 를 1개만 적용한 경우이며, 새로운 픽셀 값은 아래 수식에 의해서 결정한다.

![img3](./imgs/3.PNG)

- 경우에 따라서는 2개 혹은 그 이상의 threshold가 필요한 경우가 있고, 아래의 수식을 통해 2개의 threshold를 적용한 경우로 T1~T2를 두고 그 사이 값은 흰색, 해당 범위 밖일 경우 검은색으로 구분한다.

![img4](./imgs/4.PNG)

</br>

### 2.1.1 Threshold 설정하는 tip <a class="anchor" id="2.1.2"></a>

#### Histogram
- 가장 많이 사용되는 방식이 픽셀값의 누적 분포를 알 수 있는 Histogram 을 사용하는 것이다.

![img5](./imgs/5.PNG)

- 위의 예시 이미지를 보면, histogram에서 peak가 2개이고 간격도 충분히 떨어져 있음을 볼 수 있다. 이 경우, 대략적으로 2개의 peak 중 가운데 값으로 잘라줄 수 있다.

</br>

#### Otsu Algorithm 
- histogram 의 분포를 보았을 때, 위의 예시와 같지 않은 경우 해당 알고리즘을 적용한다.

- 통계적방법을 이용한 Otsu 알고리즘이 가장 자연스러운 threshold 값을 설정할 수 있다.

- "A threshold selection mothod from gray-level histogram" 

![img6](./imgs/6.PNG)

- 예를 들어 위의 이미지를 통해 histogram 을 구했더니 오른쪽 그림과 같이 peak가 명확하지 않은 분포를 보였다. 

- 왼쪽의 영상에서 임의의 threshold 값 **T**를 기준으로 하여, **T**보다 작은 픽셀(어두운)의 비율을 q1, 같거나 큰 픽셀의 비율을 q2라 한다. 또한 **T**를 기준으로 만들어진 2개의 그룹에 대해 평균과 표준편차를 구하고, 각각 μ1, μ2 및 σ1, σ2라고 한다.

- Otsu 알고리즘의 핵심 아이디어는 2개의 그룹에 대해 **Within class variance(그룹 내 분산)** 을 최소화 하거나 **inter class variance(그룹 간 분산)**을 최대로 하는 방향으로 **T**를 정하면, 가장 적절한 Threshold 값을 구할 수 있다. 

![img7](./imgs/7.PNG)

- '그룹 내 분산'을 최소화 하는 것과 '그룹 간 분산'을 최대화 하는 것은 같은 의미이다.

- 하지만 '그룹 간 분간' 을 구하는 식이 조금더 쉽기 때문에 이를 더 많이 사용한다. 

</br>

#### Global or Local

- 이미지 전체에 동일한 임계값을 적용하는 경우, 조명이나 특정 잡음으로 인해 문제가 발생할 수 있다. 

- 아래의 예시처럼 중심은 밝고 주변은 어두운 이미지에 대해, 각각의 영역에 대해 적절한 thresholding을 하는 경우에는 좋은 결 과를 얻을 수 있다. 

![img8](./imgs/8.PNG)

- 이처럼 이미지의 영역을 여러개로 나누고 각각에 대해 다른 threshold 값을 적용하는 방식을 **local thresholding** 이라고 부른다.

___ 

</br>


# 3. Edge 기반 segmentation 1 <a class="anchor" id="3"></a>
___

## 3.1 Edge 기반의 Segmentation <a class="anchor" id="3.1"></a>

- Edge 기반의 segmentation은 복잡하지 않고, 영역을 분리할 때 edge가 중요한 특성이 된다. 즉, 복잡하지 않고 효과가 좋아 많이 사용하지만, 영상에 noise나 부분적으로 가려지는 현상이 발생하면 성능이 매우 떨어진다. 

- Edge 기반의 segmentation은 크게 2가지 과정을 거친다.
    1. Edge 검출 : Edge pixel(Edgel)을 찾아내는 과정.
        - **Gradient, Laplacian, LoG, Canny filter** 등..
    2. Edge 연결 : Edge pixel들을 연결하는 과정
        - local processing : Gragient의 방향과 크기가 미리 정한 기준을 만족하는 경우, 인접 pixel끼리 연결.
        - global processing : Hough transform (허프 변환)
        
        </br>

### 3.1.1 Gradient <a class="anchor" id="3.1.1"></a>

- Edge 란 영상에 존재하는 불연속적인 부분을 말하며, brightness, color, texture 등의 변화가 있는 부분이다. edge 검출에는 여러가지 방법들이 있으며, **spatial filter(공간 필터)**를 많이 사용한다.

- **Spatial filter**들의 역할은 영상으로부터 gradient를 추출하며, 이 gradient는 아래의 이미지에서와 같이 가장 변화 속도가 빠른 방향을 나타내게 된다.

![img9](./imgs/9.PNG)

- 또한 gradient는 변화의 세기를 나타내는 '크기'와 가장 변화의 속도가 빠른 쪽을 나타내는 방향을 가지며 아래의 식과 같다.

![img10](./imgs/10.PNG)

</br>

### 3.1.2 Edge 검출 필터 <a class="anchor" id="3.1.2"></a>

-  Edge를 검출이란 이미지 내에서 불연속한 부분을 찾아내는 것이기 때문에 **1차 미분, 2차 미분**을 이용한다.

- 1차 미분을 사용하는 대표적인 필터는 "**Prewitt**", "**Robert**", "**Sobel**"

![img11](./imgs/11.PNG)

- 위의 수식은 Sobel filter로, x와 y 방향의 edge를 각각 검출한 후 합치는 과정을 거친다.

- 2차 미분을 사용하는 대표적인 필터는 **Laplacian** 필터가 있으며 수식은 아래와 같다.
![img12](./imgs/12.PNG)


</br>

### 3.1.3 edge 검출과 noise의 영향. <a class="anchor" id="3.1.3"></a>

- 아래의 예시처럼, noise가 많은 경우에는 아래의 그림처럼 미분을 하더라도 edge 검출이 어려울 수도 있다. 

![img13](./imgs/13.PNG)

- 그러므로 사전에 smoothing filter를 통해 잡음의 영향을 줄이는 것이 필요하다.

![img14](./imgs/14.PNG)

- 해당 예시는 동일한 데이터에 대해 가우시안 필터를 적용한 후에 edge 검출이 된 것을 보여준다.


</br>

### 3.1.4 Laplacian of Gaussian(LoG), Difference of Gaussian (DoG) <a class="anchor" id="3.1.4"></a>

- 또한 앞서 고려한 것처럼, 사전에 gaussian 필터를 적용하여 잡음을 제거하고 여기에 2차 미분 필터인 Laplacian 필터를 적용하면 아래의 예시와 같이 큰 edge 성분을 쉽게 찾아낼 수 있다.

![img15](./imgs/15.PNG)

- Gaussian, DoG, LoG 필터의 커널은 각각 아래 그림과 같다.

![img16](./imgs/16.PNG)

- LoG 는 잡음이 있는 환경에서 edge를 잘 찾을 수 있기 때문에 여러 곳에서 많이 쓰이고 있다. 

- Gaussian 필터는 σ(분산) 값에 따라 smoothing을 적용하는 범위가 달라지게 되며, SIFT(Scale Invariant Feature Transform)에서는 연산량이 많은 LoG 대신, DoG를 사용하기도 한다.


</br>

### 3.1.4 post-processing <a class="anchor" id="3.1.5"></a>

- spatial filter를 통해 검출된 edge는 바로 사용하기에 부적절하기 때문에 post-processing을 필요로 한다.

- 개체의 경계만 검출되지 않고 경계가 아닌 부분도 같이 검출되기도 하고, 경계에 해당하는 부분에서 검출된 값이 작아서 후처리 과정에서 사라지는 경우가 존재하기 때문이다. (LoG를 사용하면 경계부분이 smoothing 될 수 있음.)

- 보통 원영상에 작은 detail이 많은 경우, gradient 크기 영상에서 부터 좀더 의미있는 edge를 추출하기 위해서는 thresholding을 적용해야한다.

#### 대표적인 post-processing 

1. Thresholding : 일정 크기 이하의 edge들을 제거하여 큰 edge 위주로 정리

2. Thinning : non-maximum suppression을 적용하여, 굵은 edge를 얇게 처리

3. Hysteresis thresholding : 1 개의 threshold가 아닌 high, low, 이 두개의 threshold 값을 사용한다.
    - high threshold edge에 연결된 low threshold edge는 제거하지 않고 살려두는 것을 의미.

![img17](./imgs/17.PNG)

- 앞서 살펴본 대부분의 post-processing 과정 및 장점들은 "Canny Edge Detector"에 거의 대부분 적용된다.

___ 



</br>


# 4. Edge 기반 segmentation 2 <a class="anchor" id="4"></a>
___

## 4.1 Edge thinning <a class="anchor" id="4.1"></a>

- 검출한 edge를 1픽셀 크기로 얇게 만드는 것을 만한다.

- 보통 edge 검출하기 전에 noise를 제거하기 위해 median이나 gaussian filter와 같은 smoothing filter를 적용한다. 그 후, edge 검출을 위해 spatial filter를 적용하면, 보통은 선폭이 1픽셀 이상이기 때문에 이것을 적절한 방법으로 얇게 만들어줘야 한다.

- Thinning을 하는 이유는 얇고 선명한 (sharp edge) edge가 객체등의 응용에서 훨씬 유용하기 때문이며, Hough Transform을 이용하여 직선이나 타원 들을 검출할 때도 얇은 경우에 결과가 더 좋게 나오기 때문이며, 때로 edge가 객체의 경계에 위치하고 있는 경우는 객체의 perimeter(둘레)를 구할 때 복잡한 공식을 사용하지 않고도 쉽게 할 수 있기 때문이다. 

#### thinning은 아래 그림처럼 2종류의 방법을 사용한다.
![img18](./imgs/18.PNG)

- 4방향 또는 8방향 방법을 정했다면, 중심 픽셀의 edge 값과 주변 픽셀의 엣지값을 비교하여 중심 픽셀이 가장 큰 값인 경우는 남겨두고 그렇지 않은 경우는 제거하는 방식이다.

- 이를 non-suppression이라고도 부른다.

- 예를 들면 왼쪽 이미지에서 thinning을 하면, 맨 왼쪽/위 방향부터 3x3 sliding window를 적용시키면서, 해당 window의 중심값보다 주변값이 큰 경우 중심값을 0 으로 바꿔가는 방식이다.
![img19](./imgs/19.PNG)

</br> 

## 4.2 Canny Edge Detector <a class="anchor" id="4.2"></a>

- Canny edge detecto는 1986년에 발표되었으며, 다른 detector에 비해 성능이 뛰어나 많이 사용됨.

- optimal edge를 검출하려면 다음 3가지 항목의 특성이 좋아야한다.
    - noise 제거 또는 noise smoothing(평활화)
    - edge enhancement(개선, 강화)
    - edge localization(위치 파악)
    
- 위 3가지 항목의 측성을 살펴보기 ㅡ위해 아래 그림과 같은 이상적인 edge에 white gaussian noise가 추가된 경우를 보면,

![img20](./imgs/20.PNG)

- noise가 있더라도 우수한 edge 검출력을 보이려면, 실제로 edge가 위치하고 있는  x = 0 에 noise보다 큰 응답 특성(responce)가 나와야하고, 실제로 가장 큰 edge 값이 나와야한다. (Noise smoothing, Edge localization)

![img21](./imgs/21.PNG)

- 또한 x = 0 근처에서 단 하나의 최고값만을 갖도록 해야한다. 아래의 이미지에서 처럼, 다음 peak와는 어느 정도 거리 확보가 필요하다. (edge enhancement)

![img22](./imgs/22.PNG)

- Canny edge detector는 위 3가지 특성을 극대화하기 위한 목적으로 개발되었고, 다음과 같은 3가지 과정으로 이루어져 있다.
    1. Gaussian smoothing을 한 후에 Gradient의 크기와 방향을 구한다.
    2. Non-maximum suppression을 통해 검출된 edge를 1픽셀의 크기로 얇게 만든다.
    3. Edge linking 및 Hysteresis thresholding을 통해 의미있는 edge만 남긴다.

</br>

### 4.2.1 Canny Edge Detector의 상세과정 - Smoothing, Gradient 구하기 <a class="anchor" id="4.2.1"></a>

- 첫 번째 과정은 noise의 영향을 최소화하기 위해 gaussian filter를 적용하여 영상을 부드럽게 만들고, gradient를 구하기 위해 x 및 y 방향으로 미분하는 것이다.

- 보통 gaussian filtering을 먼저하고 미분을 하지만 gaussian 함수의 convolution 특성 상 gaussian 함수의 미분을 먼저 한 뒤에 convolution을 해도 상관이 없다. 이는 아래의 수식과 같다 (I = edge를 검출할 원 영상)

![img23](./imgs/23.PNG)

- gaussian 함수를 미분한 수식은 아래와 같고, 이 식에서 **σ**는 연산을 적용할 kernel의 크기를 결정한다. 

![img24](./imgs/24.PNG)

- 위의 식에서 scale(**σ**)는 smoothing의 정도를 결정하는 변수이고, 이 값이 커질수록 아래와 같은 성질들을 갖는다.
    - noise edge를 더 많이 제거할 수 있다.
    - edge를 더 부드럽게 하면서 두껍게 만든다.
    - 영상에서 섬세한 detail들이 제거된다.
    
- 위의 성질들을 고려하여, smoothing된 영상에서 x, y 방향의 미분을 구하면 아래의 이미지와 같다.

![img25](./imgs/25.PNG)

- 이 결과를 이용해 gradient의 크기와 방향을 구하고, 구한 edge를 thresholding을 통해 해당 threshold 이상의 값만 보면 아래와 같은 결과가 나온다.

![img26](./imgs/26.PNG)

</br>

### 4.2.2 Canny Edge Detector의 상세과정 - Thinning (non-maximum suppression) <a class="anchor" id="4.2.2"></a>

- Canny edge 검출에서의 thinning은 앞선 과정들과 조금 다르다. 첫 번째 과정을 통해 gradient의 방향을 구했기 때문에 gradient의 변화의 방향을 이미 알고 있으므로, 변화하는 방향으로 스캔을 하면서 최대값이 되는 부분을 찾는다.

- 아래의 왼쪽 그림과 같이 gradient magnitude 영상을 보았을 때, gradient 방향이 오른쪽 직선과 같은 방향이라면, 직선 방향으로 스캔을 하면 가장 큰 값만 남기고 픽셀 값을 모두 0으로 만든다. 이를 통해 1픽셀 단위의 얇은 선이 만들어지게 된다.

![img27](./imgs/27.PNG)

- 앞서 구했던 영상에 대해 thinning을 적용하면, 아래 그림과 같이 얇은 선을 얻을 수 있으며, 이전과 동일하게 thresholding을 한 결과도 아래와 같음을 볼 수 있다.

![img28](./imgs/28.PNG)


</br>

### 4.2.3 Canny Edge Detector의 상세과정 - Hysteresis <a class="anchor" id="4.2.3"></a>

- Hysteresis 는 2개의 threshold 값을 적용하여, high threshold edge에 연결된 low threshold edge는 제거하지 않고 살려 두는 것.

- 아래의 그림은 hysteresis 예시이며, strong edge에 연결된 weak edge들이 적절하게 살아났음을 볼 수 있다. 

![img29](./imgs/29.PNG)

</br>

### 4.2.4 Canny Edge 검출에서 **σ** 의 영향 <a class="anchor" id="4.2.4"></a>

- 아래의 이미지는 scale의 변화에 따라 검출되는 edge의 모양이 달라지는 것을 보여준다. **σ** 값이 클수록 strong edge가 검출되고, 반면에 **σ** 값이 작을수록 섬세한 특징(weak edge)을 보기 좋다. 

![img30](./imgs/30.PNG)

- 또한 **σ** 값이 커지면 edge 검출은 잘 되지만, edge의 위치가 smoothing에 의해 달라질 수 있기 때문에 edge 검출과 정확도에 trade-off 관계가 있다는 것을 주의해야 한다.




</br> 

## 4.3 SUSAN Edge Detector <a class="anchor" id="4.3"></a>

- SUSAN(Smallest Univalue Segment Assimilating Nucleus) edge detector는 1995년에 발표되었으며, Canny detector와 같이 성능이 뛰어나 많이 사용됨.

- low-level 영상처리에 대한 새로운 접근법 제시. (edge 검출 성능 뛰어나고 noise 제거도 잘 됨.)
    
- 대부분의 edge 검출 알고리즘은 derivative, gradient 등을 사용하지만 SUSAN은 이를 사용하지 않는다. 또한 다른 대부분의 영상처리 알고리즘들이 3x3 or 5x5 와 같은 정방형 window 혹은 mask를 사용하지만 SUSAN에서는 원형 또는 원형에 근접한 마스크를 사용한다. 

- SUSAN 에서 USAN(Univalue Segment Assimilating Nucleus)를 이해해야 하는데, 여기서 Nucleus는 mask 중심에 있는 픽셀 값을 의미한다. 

- 아래의 그림은 논문에 제시된 그림으로, 여기서 원은 mask 영역을 나타내고 '+' 부호의 위치에 있는 픽셀이 mask의 중심(Nucleus)가 된다.

![img31](./imgs/31.PNG)

- light area와 dark area로 구성된 위 그림에 5 종류의 mask 형태를 보이고 있다. 여기서 '**e**'는 Nucleus와 주변 픽셀이 모두 같은 경우로, edge나 corner가 아니지만, 나머지 '**a ~ d**' 는 edge나 corner가 mask에 포함이 된 경우이다. 

- USAN은 전체 마스크에서 Nucleus와 같은 (혹은 거의 비슷한) 값을 갖는 면적을 뜻한다. 이 면적을 보면, 해당 mask가 평탄한 지역에 있는지, edge 나 corner에 있는지 파악이 가능하다.

![img32](./imgs/32.PNG)

- 다음 그림은 위 그림에서 USAN 영역에 해당되는 부분은 나타낸 이미지이다. Nucleus와 같은 부분은 흰색으로, 다른 부분은 검은색으로 표시를 한 것이다.

- 이때, edge 나 corner를 검출하기 위해 해당 식이 적용된다. 

![img33](./imgs/33.PNG)

1. r0 벡터 : Nucleus , r벡터 : mask 영역에 있는 다른 픽셀의 위치
2. I : 픽셀의 intensity(밝기)
3. t : threshold(유사도)

- 즉, 위 식의 의미는 **중심에 있는 픽셀의 intensity와 threshold(t) 이내로 비슷한 경우는 1이 되고, 그 이상의 차이를 보이는 경우는 0 이 되는 비선형 필터이다.**

- **c** 는 결과적으로 전체 mask 내에서 중심과 threshold 범위에 있는 픽셀의 개수를 의미한다.

- SUSAN edge detector에서 사용되는 mask는 반지름이 3.4를 많이 사용하며, 이 경우에 마스크에 있는 픽셀은 37개이다. 최소 마스크의 크기는 3x  3

</br> 

___
## 5. Region base Segmentation <a class="anchor" id="5"></a>

___

- edge 기반의 Segmentation은 영상에서의 difference(차이)가 나는 부분에 집중을 하였다면, region base Segmentation 방법은 영상에서 비슷한 속성을 갖는 부분(similarity)에 집중하는 방식이며, inside-out 방식이라고 볼 수 있다.

- 여기서 말하는 비슷한 속성이란 intensity, gray-level 과 같은 밝기 뿐만 아니라, texture(표면의 무늬), color 등도 해당된다.

![img35](./imgs/35.PNG)

- 위의 그림 처럼 극단적인 영상에서 segmentation을 진행 한 예가 아래의 그림이다. 

![img36](./imgs/36.PNG)

- Canny edge detector로 edge를 검출하는 경우와 Region base 방식들 중 평균과 표준 편차를 이용해 구별하는 경우의 그림이다. 


</br> 

### 5.1 Region을 정하는 방법 <a class="anchor" id="5.1"></a>

- region base Segmentation 에서 region을 결정하는 방식은 매우 다양하다.

    - Region Growing
    - Region Merging
    - Region Splitting
    - Split and Merge
    - Watershed
    - ...

</br> 

### 5.1.1 Region Growing <a class="anchor" id="5.1.1"></a>

-  가장 많이 사용되는 방식으로, 기준 픽셀을 정하고 이와 비슷한 속성을 갖는 픽셀로 region을 확장하고 같은 속성을 갖는 것들이 없으면 확장을 마치는 방식.

![img37](./imgs/37.PNG)

- Region Growing에서 가장 중요한 seed(시작 픽셀)를 정하는 방식
    1. 사전에 seed 위치 지정
    2. 모든 픽셀을 seed로 가정
    3. 무작위로 seed 위치를 지정
    
- seed 픽셀로부터 영역을 확장하는 방식
    1. 원래의 seed 픽셀과 비교하여 일정 범위 이내일 때만 region을 확장하는 방식. (seed를 어느것으로 잡느냐에 따라 결과가 달라짐)
    2. 확장된 위치의 픽셀과 비교 : 원래의 seed 픽셀의 위치가 아니라 영역이 커지면 비교할 픽셀의 위치가 커지는 방향으로 바뀌는 방식.
        - 이 방식은 seed 와 멀리 있는 픽셀은 값이 차이가 많이 나도 같은 영역으로 처리하는 문제발생가능.. (심각한 drift 현상)
        
    3. region의 통계적 특성과 비교 : 약간의 drift 있을 수 있지만 안전.
    
- 처리속도가 빠르고 개념적으로 단순, 동시에 여러개의 기준을 설정할 수 있다.

- 하지만 영상을 전체적으로 보는 것이 아닌 local method 이다. 또한 noise에 민감하고 drift 현상이 단점.

<a class="anchor" id="2"></a>

___ 

</br>
